Now that we have collected a lot of data, let's try to combine it all and see if it's useful in prediction! Let's assume that the task is nowcasting, that we are given data up to this year and have to predict the refugee traffic of the current year.

First, let's load in the data sources

In [847]:
import pandas as pd
import numpy as np

path = '../../data/clean/'

# data from geraldine
unhcr = pd.read_csv(path + 'unhcr.csv', engine='pyarrow')

# yearly sources from the internet
idp = pd.read_csv(path + 'idp.csv', engine='pyarrow')
fsi = pd.read_csv(path + 'fragility_indices.csv', engine='pyarrow')
migration = pd.read_csv(path + 'migration_stocks.csv')

# monthly sources from hannes + gdelt team
conflict = pd.read_csv(path + 'mueller_conflict_forecasts.csv')
lda = pd.read_csv(path + 'lda.csv')
gdelt = pd.read_csv(path + 'gdelt.csv', engine='pyarrow')

trends_path = '../../notebooks/trends/data/'
# monthly trends data
# semantic_trends = pd.read_csv(trends_path + 'semantic_topic_trends.csv')
# main_cities_trends = pd.read_csv(trends_path + 'city_topic_trends_1.csv')
# countries_trends = pd.read_csv(trends_path + 'country_topic_trends_1.csv')
# border_cities_trends = pd.read_csv(trends_path + 'bordering_cities_trends.csv')

That's a lot of sources! First thing I think of is we need to convert conflict, lda, and gdelt to a yearly format.

There are multiple ways to do this. A simple approach is to summarize and do boxplots of the months, aggregating to yearly data. 

Here we have the function agg_year, which produces a boxplot, but also includes the log difference of the data, to represent the average log change for a given year.

In [848]:
# Define the agg_year function
import numpy as np
import pandas as pd

def avg_log_diff(series):
    return np.log1p(series).diff().mean()

# Define the agg_year function
def agg_year(series):
    # Calculate the statistics using numpy
    minimum = np.min(series)
    q25 = np.percentile(series, 25)
    median = np.median(series)
    q75 = np.percentile(series, 75)
    maximum = np.max(series)
    log_diff = avg_log_diff(series)
    
    # Return the statistics as a list
    return [minimum, q25, median, q75, maximum, log_diff]

def split_nested_list(series):
    df = pd.DataFrame(series.apply(pd.Series).values.tolist())
    df.columns = series.name + '_' + pd.Series(['min','25th','mean','75th','max', 'log_diff'])
    return df

def split_df(df):
    new_df = pd.concat([split_nested_list(df[col]) for col in df.columns], axis=1)
    new_df.index = df.index
    return new_df

This is another way to aggregate years where we convert a single year into 12 columns: 4 quarters and 4 summary statistics (min, mean, max, and pct change of the mean).

In [849]:
contflict_original = conflict.copy()

In [855]:
contflict_original[(contflict_original.isocode == 'AFG') & (contflict_original.year == 2015)]

,Unnamed: 0,isocode,year,month,ons_armedconf12_pred_jut_x,ons_armedconf12_pred_top_x,ons_armedconf12_pred_aug_x,ons_armedconf12_pred_jut_y,ons_armedconf12_pred_top_y,ons_armedconf12_pred_aug_y,lnbest_pc_12_pred_jut,lnbest_pc_12_pred_top,lnbest_pc_12_pred_aug
33336,33336,AFG,2015,1,0.919407,0.387879,0.836896,0.916742,0.352571,0.838462,0.026720,0.010795,0.022456
33522,33522,AFG,2015,2,0.922114,0.371774,0.843205,0.916571,0.353100,0.839948,0.022067,0.011012,0.021697
33708,33708,AFG,2015,3,0.919411,0.366943,0.856330,0.914071,0.345406,0.843037,0.021767,0.011829,0.022391
33894,33894,AFG,2015,4,0.918707,0.381615,0.844632,0.913551,0.358489,0.843362,0.022114,0.012206,0.022488
34081,34081,AFG,2015,5,0.922377,0.378289,0.853824,0.912550,0.347768,0.839890,0.024925,0.012511,0.023976
34268,34268,AFG,2015,6,0.918196,0.378064,0.846904,0.919028,0.346064,0.837269,0.023664,0.011901,0.022373
34455,34455,AFG,2015,7,0.915169,0.379062,0.854396,0.920137,0.348637,0.844686,0.022655,0.011993,0.022259
34642,34642,AFG,2015,8,0.921259,0.374455,0.842280,0.918432,0.351498,0.831068,0.025347,0.016208,0.023768
34829,34829,AFG,2015,9,0.913073,0.369887,0.844030,0.915840,0.352828,0.841021,0.026339,0.012504,0.023302
35016,35016,AFG,2015,10,0.915244,0.413077,0.856743,0.912265,0.384284,0.847151,0.027406,0.015183,0.023823


In [850]:
# apply this to conflict data
conflict = conflict.groupby(['isocode', 'year'])[['ons_armedconf12_pred_jut_x', 'ons_armedconf12_pred_top_x', 'ons_armedconf12_pred_aug_x', 
                                                  'ons_armedconf12_pred_jut_y', 'ons_armedconf12_pred_top_y', 'ons_armedconf12_pred_aug_y', 
                                                  'lnbest_pc_12_pred_jut', 'lnbest_pc_12_pred_top', 'lnbest_pc_12_pred_aug']].agg(agg_year)

conflict = split_df(conflict)

In [857]:
conflict.reset_index()[(conflict.reset_index().isocode == 'AFG') & (conflict.reset_index().year==2015)]

,isocode,year,ons_armedconf12_pred_jut_x_min,ons_armedconf12_pred_jut_x_25th,ons_armedconf12_pred_jut_x_mean,ons_armedconf12_pred_jut_x_75th,ons_armedconf12_pred_jut_x_max,ons_armedconf12_pred_jut_x_log_diff,ons_armedconf12_pred_top_x_min,ons_armedconf12_pred_top_x_25th,...,lnbest_pc_12_pred_top_mean,lnbest_pc_12_pred_top_75th,lnbest_pc_12_pred_top_max,lnbest_pc_12_pred_top_log_diff,lnbest_pc_12_pred_aug_min,lnbest_pc_12_pred_aug_25th,lnbest_pc_12_pred_aug_mean,lnbest_pc_12_pred_aug_75th,lnbest_pc_12_pred_aug_max,lnbest_pc_12_pred_aug_log_diff
15,AFG,2015,0.913073,0.917015,0.918516,0.919873,0.922377,-0.000051,0.366943,0.373785,...,0.012355,0.014401,0.021082,0.000921,0.021697,0.022345,0.022472,0.023418,0.023976,0.000054


In [813]:
# now the lda data
lda = lda.groupby(['isocode','year'])[['ste_theta0', 'ste_theta1', 'ste_theta2', 'ste_theta3',
       'ste_theta4', 'ste_theta5', 'ste_theta6', 'ste_theta7', 'ste_theta8',
       'ste_theta9', 'ste_theta10', 'ste_theta11', 'ste_theta12',
       'ste_theta13', 'ste_theta14', 'ste_theta0_stock', 'ste_theta1_stock',
       'ste_theta2_stock', 'ste_theta3_stock', 'ste_theta4_stock',
       'ste_theta5_stock', 'ste_theta6_stock', 'ste_theta7_stock',
       'ste_theta8_stock', 'ste_theta9_stock', 'ste_theta10_stock',
       'ste_theta11_stock', 'ste_theta12_stock', 'ste_theta13_stock',
       'ste_theta14_stock', 'sentiment_stock']].agg(agg_year)
lda = split_df(lda)

In [814]:
gdelt = gdelt.groupby(['isocode','year'])[['count_events_1',
       'count_events_2', 'count_events_3', 'count_events_4', 'count_events_5',
       'count_events_6', 'count_events_7', 'count_events_8', 'count_events_9',
       'count_events_10', 'count_events_11', 'count_events_12',
       'count_events_13', 'count_events_14', 'count_events_15',
       'count_events_16', 'count_events_17', 'count_events_18',
       'count_events_19', 'count_events_20']].agg(agg_year)
gdelt = split_df(gdelt)

In [815]:
an_index = unhcr[['iso_o','iso_d','year','Id']].copy()
columns_to_drop = ['Country_o','Country_d']
unhcr.drop(columns_to_drop, axis=1, inplace=True)

This is a mapper to get the continent for a given country.

In [816]:
import pycountry_convert as pc

def country_to_continent(iso3):
    if iso3 == 'UVK':
        return 'EU'
    elif iso3 =='TLS':
        return 'OC'
    elif iso3 =='WBG':
        return 'AS'

    country_alpha2 = pc.country_alpha3_to_country_alpha2(iso3)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    return country_continent_code

def mapper(series, converter):
    unique_keys = series.drop_duplicates()
    unique_vals = unique_keys.apply(converter)
    mapper_dict = dict(zip(unique_keys, unique_vals))
    series = series.map(mapper_dict)
    series.name = series.name + '_continent'
    return series

In [817]:
continents = unhcr[['iso_o','iso_d']].apply(lambda x: mapper(x, country_to_continent)).rename({'iso_o':'iso_o_continent','iso_d':'iso_d_continent'}, axis=1)
unhcr = pd.concat([continents, unhcr], axis=1)

Let's merge the data with the other yearly sources

In [818]:
unhcr_extra = unhcr.merge(idp.drop('', axis=1), on=['iso_o', 'year'], how='left') \
    .merge(fsi, left_on=['iso_o', 'year'], right_on=['iso', 'year'], how='left') \
    .drop('iso', axis=1) \
    .merge(fsi, left_on=['iso_d', 'year'], right_on=['iso', 'year'], how='left', suffixes=('_o','_d')) \
    .drop('iso', axis=1) \
    .merge(migration, on=['year','iso_o','iso_d'])

And now let's merge with the other monthly sources that have been aggregated

In [819]:
unhcr_extra = unhcr_extra.merge(conflict, how='left',left_on=['iso_o','year'], right_on=['isocode','year']) \
    .merge(lda, how='left', left_on=['iso_o','year'], right_on=['isocode','year']) \
    .merge(gdelt,  how='left', left_on=['iso_o','year'], right_on=['isocode','year'])

The additional dataframes take up a lot of space and slow down computation time, so let's delete them.

In [820]:
import gc

del gdelt; del unhcr; del lda; del conflict; del fsi; del idp; del migration

gc.collect()

0

And now, let's create a lagged features, in this case let's do 5 years of lagged features.

In [821]:
from multiprocesspandas import applyparallel

def multi_shift(df, shift_range, columns):
    shifted_data = [df[columns].shift(shift_value) for shift_value in range(shift_range.start, shift_range.stop)]
    shifted_df = pd.concat(shifted_data, axis=1, keys=[f'Shift_{shift_value}' for shift_value in range(shift_range.start, shift_range.stop)])
    shifted_df.columns = shifted_df.columns.map(lambda col: '_'.join(col).strip())
    return shifted_df

shift_cols = ['newarrival','dead_o','fsi_overall_o','dead_d','ons_armedconf12_pred_top_x_mean', 'ons_armedconf12_pred_top_x_75th', 'ons_armedconf12_pred_top_x_log_diff']
# shift_cols = ['newarrival','dead_o','fsi_overall_o']

shifted_features = unhcr_extra[shift_cols].copy()
shifted_features = pd.concat([shifted_features, an_index], axis=1).set_index(an_index.columns.tolist()).groupby('Id', group_keys=False).apply(lambda x: multi_shift(x, range(1,3), shift_cols))

In [822]:
unhcr_extra = pd.concat([unhcr_extra, shifted_features.reset_index(drop=True)], axis=1)

In [823]:
del shifted_features
gc.collect()

0

In [824]:
unhcr_extra

,iso_o_continent,iso_d_continent,iso_o,iso_d,year,pop_o,CPI_o,GDP_PP_o,GDP_PPP_o,island_o,...,Shift_1_ons_armedconf12_pred_top_x_mean,Shift_1_ons_armedconf12_pred_top_x_75th,Shift_1_ons_armedconf12_pred_top_x_log_diff,Shift_2_newarrival,Shift_2_dead_o,Shift_2_fsi_overall_o,Shift_2_dead_d,Shift_2_ons_armedconf12_pred_top_x_mean,Shift_2_ons_armedconf12_pred_top_x_75th,Shift_2_ons_armedconf12_pred_top_x_log_diff
0,AS,EU,AFG,ALB,2000,19.500,-3042.840501,1342.77,0.036,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AS,EU,AFG,ALB,2001,19.700,-2103.557431,1392.77,0.036,0,...,0.552818,0.609259,-0.007045,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AS,EU,AFG,ALB,2002,18.707,649.220570,1442.77,0.036,0,...,0.554766,0.594786,0.003992,4.0,1.0,NaN,0.0,0.552818,0.609259,-0.007045
3,AS,EU,AFG,ALB,2003,19.477,6.530000,1506.22,0.037,0,...,0.547578,0.574180,-0.002869,0.0,1.0,NaN,0.0,0.554766,0.594786,0.003992
4,AS,EU,AFG,ALB,2004,20.237,13.266000,1459.35,0.036,0,...,0.499888,0.508043,-0.000466,0.0,1.0,NaN,0.0,0.547578,0.574180,-0.002869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955495,AF,AF,ZWE,ZMB,2020,15.189,348.586000,2048.27,0.025,0,...,0.161706,0.187285,-0.005174,2.0,0.0,102.293753,0.0,0.238051,0.250770,-0.004368
955496,AF,AF,ZWE,ZMB,2021,15.492,60.737000,2151.97,0.025,0,...,0.120348,0.129432,-0.000880,0.0,0.0,99.500000,0.0,0.161706,0.187285,-0.005174
955497,AF,AF,ZWE,ZMB,2022,15.817,547.255000,2171.10,0.025,0,...,0.071631,0.095867,-0.005527,1.0,0.0,99.220857,0.0,0.120348,0.129432,-0.000880
955498,AF,AF,ZWE,ZMB,2023,16.164,100.000000,2184.84,0.025,0,...,0.064773,0.079502,0.000591,0.0,0.0,99.100000,0.0,0.071631,0.095867,-0.005527


In [843]:
unhcr_extra[unhcr_extra.ons_armedconf12_pred_top_x_max.isna() & (unhcr_extra.year == 2015)].drop_duplicates(['iso_o']).iso_o

14640     AND
39015     ABW
360765    HKG
443640    UVK
497265    MAC
550890    FSM
589890    NRU
682515    PRI
711765    SMR
882390    TUV
936015    WBG
Name: iso_o, dtype: object

In [825]:
(unhcr_extra[~unhcr_extra.newarrival.isna()& ~unhcr_extra['Shift_2_newarrival'].isna() & ~unhcr_extra['Shift_2_fsi_overall_o'].isna()].isna().sum()/unhcr_extra[~unhcr_extra.newarrival.isna()& ~unhcr_extra['Shift_2_newarrival'].isna() & ~unhcr_extra['Shift_2_fsi_overall_o'].isna()].shape[0]).nlargest(20)

Shift_2_ons_armedconf12_pred_top_x_mean        0.057948
Shift_2_ons_armedconf12_pred_top_x_75th        0.057948
Shift_2_ons_armedconf12_pred_top_x_log_diff    0.057948
Shift_1_ons_armedconf12_pred_top_x_mean        0.057584
Shift_1_ons_armedconf12_pred_top_x_75th        0.057584
Shift_1_ons_armedconf12_pred_top_x_log_diff    0.057584
ons_armedconf12_pred_jut_x_min                 0.057219
ons_armedconf12_pred_jut_x_25th                0.057219
ons_armedconf12_pred_jut_x_mean                0.057219
ons_armedconf12_pred_jut_x_75th                0.057219
ons_armedconf12_pred_jut_x_max                 0.057219
ons_armedconf12_pred_jut_x_log_diff            0.057219
ons_armedconf12_pred_top_x_min                 0.057219
ons_armedconf12_pred_top_x_25th                0.057219
ons_armedconf12_pred_top_x_mean                0.057219
ons_armedconf12_pred_top_x_75th                0.057219
ons_armedconf12_pred_top_x_max                 0.057219
ons_armedconf12_pred_top_x_log_diff            0

That's a lot of columns! We'll probably need to apply pca before working with the data, which means we'll need to scale the numerical data. We will treat the categorical data separately. The categorical data includes:
- Binary encoded idp data
- Continent origin and destination
- Country origin and destination

In [710]:
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.decomposition import PCA
# from ppca import PPCA
from sklearn.pipeline import make_pipeline, Pipeline

be = BinaryEncoder()
cont_scaler = RobustScaler()
imputer = IterativeImputer(max_iter=5,
                 initial_strategy = 'most_frequent',
                 verbose=True,
                 estimator=ExtraTreesRegressor(n_estimators=20,
                                               min_samples_leaf=10,
                                               min_samples_split=10,
                                               random_state=0,
                                               n_jobs=-1))
# pca = PCA(n_components=.95)

# cont_pipe = make_pipeline(
#     cont_scaler#,
#     # ppca
# )

binary_cols = ['iso_o','iso_d','iso_o_continent','iso_d_continent', 'Id']
ignore_cols = ['conflict_stock_idp_pop_0',
       'conflict_stock_idp_pop_1', 'conflict_stock_idp_pop_2',
       'conflict_stock_idp_pop_3', 'conflict_stock_idp_pop_4',
       'conflict_idp_pop_0', 'conflict_idp_pop_1', 'conflict_idp_pop_2',
       'conflict_idp_pop_3', 'conflict_idp_pop_4']
numerical_cols = list(set(unhcr_extra.columns) - set(binary_cols + ignore_cols))

from sklearn.compose import ColumnTransformer

transform_cols = ColumnTransformer(
    [('cat', be, binary_cols)],
    # ('numerical', cont_pipe, numerical_cols)],
     remainder='passthrough')

impute_transformer = make_pipeline(
    transform_cols, 
    imputer
)

In [711]:
impute_transformer.fit_transform(sa_X)

[IterativeImputer] Completing matrix with shape (1826, 480)
[IterativeImputer] Change: 0.00015128920874152963, scaled tolerance: 8511.92 
[IterativeImputer] Early stopping criterion reached.


array([[ 0.        ,  0.        ,  0.        , ...,  0.06391608,
         0.07670197,  0.00312411],
       [ 0.        ,  0.        ,  0.        , ...,  0.06290559,
         0.07709196, -0.0019028 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.07334637,
         0.08724441, -0.0015743 ],
       ...,
       [ 1.        ,  1.        ,  0.        , ...,  0.22716175,
         0.24485088,  0.00527368],
       [ 1.        ,  1.        ,  0.        , ...,  0.18636078,
         0.19798908, -0.00244298],
       [ 1.        ,  1.        ,  0.        , ...,  0.21069288,
         0.22130984,  0.00163377]])